<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_tagging/scat_footprint/scat_footprint_train_yolov4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training YOLOv4 to detect scat and footprints from EOL images
---
*Last Updated 3 June 2021*   
-Runs in Python 3-
Use images with annotations to train YOLOv4 to detect scat and footprints from EOL images.

Datasets were downloaded to Google Drive in [scat_footprint_preprocessing.ipynb](https://github.com/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_tagging/scat_footprint/scat_footprint_preprocessing.ipynb). 

**YOLOv4 was trained for 1200 epochs ('iterations').** Scat/footprint object detection models never learned despite adjusting augmentation and model hyperparameters for many training sessions. If successful approaches are found at a later date, steps for adding tags to images will be included. Custom anchor boxes were used to optimize coverage for the dataset and image augmentation was used to increase dataset size from 500 img per class to 1000 img, but loss never decreased below 5 and final mAP was <25%.

Notes:   
* Before you you start: change the runtime to "GPU" with "High RAM" 
* Change filepaths/taxon names where you see 'TO DO'   

References:   
* [AlexeyAB's Darknet training instructions - GitHub](https://github.com/AlexeyAB/darknet)   
* [PJReddie's Darknet training instructions - GitHub](https://github.com/pjreddie/darknet)
* [PJReddie's Darknet training instructions - pjreddie.com](https://pjreddie.com/darknet/yolo/)

## Installs & Imports
---

In [ ]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Install libraries
# Make sure you are using Python 3.6
!python --version
!pip install cython
!pip install opencv-python

import os
import pathlib
import shutil 
import glob

/content/drive/My Drive/train
Python 3.7.10
     |███████████████▊                | 202.7MB 1.3MB/s eta 0:02:37

## Model preparation (only run once)

In [ ]:
# Install darknet
!git clone https://github.com/AlexeyAB/darknet

# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile



# TO DO: Type in the path to your working directory in form field to right
basewd = "/content/drive/MyDrive/train" #@param {type:"string"}
%cd $basewd

# Download darkflow (the tensorflow implementation of YOLO)
if os.path.exists("darknet"):
    %cd darknet

elif not os.path.exists("darknet"):
    !git clone https://github.com/AlexeyAB/darknet
    # Compile darkflow
    %cd darknet
    !python setup.py build_ext --inplace
    # Change makefile to have GPU and OPENCV enabled
    !sed -i 's/OPENCV=0/OPENCV=1/' Makefile
    !sed -i 's/GPU=0/GPU=1/' Makefile
    !sed -i 's/CUDNN=0/CUDNN=1/' Makefile
    !sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

wd = 'darknet'
%cd $wd

# Verify CUDA version (for using GPU)
!/usr/local/cuda/bin/nvcc --version

# Make darknet
!make

### Train/test dataset preparation (only run once)
Prepare datasets and make custom anchor box dimensions to help the model learn from image regions based on scat/footprint bounding boxes. Anchor boxes are the regions CNNs use to scan for objects and their dimensions can influence detection results.

In [ ]:
# Copy test images and annotations to darknet
%cd ../
!cp test_images/* darknet/build/darknet/x64/data/obj
!cp test_ann/* darknet/build/darknet/x64/data/obj

print("Number of test images copied to darknet: ")
!ls test_images | wc -l
print("\nNumber of test annotations copied to darknet: ")
!ls test_ann | wc -l
print("\nNumber of test dataset files in darknet: ")
!ls darknet/build/darknet/x64/data/obj | wc -l

In [ ]:
# Copy train images and annotations to darknet

!cp images/* darknet/build/darknet/x64/data/obj
!cp annotations/* darknet/build/darknet/x64/data/obj

print("Number of train images copied to darknet: ")
!ls images | wc -l
print("\nNumber of train annotations copied to darknet: ")
!ls annotations/  | wc -l
print("\nNumber of train dataset files in darknet: ")
!ls darknet/build/darknet/x64/data/obj | wc -l

In [ ]:
# Delete tf.record files from copied train and test images (leftover from tensorflow object detection)
path = "build/darknet/x64/data/obj"

files = os.listdir(path)
flist = [file for file in files if file.endswith(".record")]
for file in flist:
  fpath = os.path.join(path, file)
  os.remove(fpath)

In [ ]:
# Convert xml annotations to YOLOv4 format
%cd darknet/build/darknet/x64/data/obj

!python xml_to_yolo.py

!rm -r ./*.xml

%cd ../
!mv labels/xml_to_yolo.py ..

In [ ]:
# Make train.txt and test.txt files that list images needed by YOLOv3 for training
%cd $wd
%cd ../

# Train dataset
path = "images"
with open('train.txt', 'w', encoding='utf-8') as f:
  for dir, dirs, files in os.walk(path):
    files = [fn for fn in files]
    for fn in files:
      fn = "build/darknet/x64/data/obj/" + fn
      f.writelines(fn + '\n')

# Test dataset
path = "test_images"
with open('test.txt', 'w', encoding='utf-8') as f:
  for dir, dirs, files in os.walk(path):
    files = [fn for fn in files if not fn.endswith('.record')]
    for fn in files:
      fn = "build/darknet/x64/data/obj/" + fn
      f.writelines(fn + '\n')

In [ ]:
# Generate custom anchor box dimensions based on annotation sizes in training dataset
# Copy these values to cfg files for training to improve accuracy
%cd $wd

!python build/darknet/x64/gen_anchors.py -filelist build/darknet/x64/data/train.txt -output_dir ./ -num_clusters 5

## Train the model
---

In [ ]:
# Build darknet during each training session, even if model and datasets were made in previous sessions

# TO DO: Type in the path to your working directory in form field to right
wd = "/content/drive/MyDrive/train/darknet" #@param {type:"string"}
%cd $wd

# Verify CUDA version (for using GPU)
!/usr/local/cuda/bin/nvcc --version

# Make darknet
!make

# Copy original model config file to yolo-obj.cfg to use as a working file 
# Double click the cfg file in Colab's file explorer to open Colab's text editor
# You can adjust hyperparameters and anchor box dimensions
!cp cfg/yolov4-custom.cfg cfg/yolo-obj.cfg

In [ ]:
# Actual training
# add the flag -clear if retraining from scratch, otherwise re-running will build on previous training steps

!./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/yolov4.conv.137 -map -dont_show 

Streaming output truncated to the last 5000 lines.

You use flag -show_imgs, so will be saved aug_...jpg images. Click on window and press ESC button 

You use flag -show_imgs, so will be saved aug_...jpg images. Click on window and press ESC button 

You use flag -show_imgs, so will be saved aug_...jpg images. Click on window and press ESC button 

You use flag -show_imgs, so will be saved aug_...jpg images. Click on window and press ESC button 

You use flag -show_imgs, so will be saved aug_...jpg images. Click on window and press ESC button 

You use flag -show_imgs, so will be saved aug_...jpg images. Click on window and press ESC button 

You use flag -show_imgs, so will be saved aug_...jpg images. Click on window and press ESC button 

You use flag -show_imgs, so will be saved aug_...jpg images. Click on window and press ESC button 

You use flag -show_imgs, so will be saved aug_...jpg images. Click on window and press ESC button 

You use flag -show_imgs, so will be saved aug_..